# Datos de partido equipos

In [10]:
pip install nba_api

3764.96s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2
Note: you may need to restart the kernel to use updated packages.


In [454]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder, teamgamelog
from nba_api.stats.static import teams
from datetime import datetime, timedelta
import warnings


In [610]:
def get_games(season):
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
    games = gamefinder.get_data_frames()[0]
    games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
    
    date_thresholds = {
        '2019-20': '2019-10-22',
        '2020-21': '2020-12-22',
        '2021-22': '2021-10-19',
        '2022-23': '2022-10-18',
        '2023-24': '2023-10-24'
    }
    if season in date_thresholds:
        date_threshold = pd.to_datetime(date_thresholds[season])
        games = games[games['GAME_DATE'] > date_threshold]
    
    return games

def reformat_dataframe(df):
    
    # Create new columns for home and away team IDs and names
    # Extract relevant fields for each GAME_ID
    df['HOME_TEAM_ID'] = df.apply(lambda row: row['TEAM_ID'] if '@' in row['MATCHUP'] else row['TEAM_ID'] if ' vs. ' in row['MATCHUP'] and row['MATCHUP'].split(' vs. ')[0] == row['TEAM_ABBREVIATION'] else None, axis=1)
    df['VISITOR_TEAM_ID'] = df.apply(lambda row: row['TEAM_ID'] if ' vs. ' in row['MATCHUP'] and row['MATCHUP'].split(' vs. ')[1] == row['TEAM_ABBREVIATION'] else row['TEAM_ID'] if '@' in row['MATCHUP'] and row['MATCHUP'].split(' @ ')[0] == row['TEAM_ABBREVIATION'] else None, axis=1)
    
    # Now apply the same logic to get team names
    df['HOME_TEAM_NAME'] = df.apply(lambda row: row['TEAM_NAME'] if '@' in row['MATCHUP'] else row['TEAM_NAME'] if ' vs. ' in row['MATCHUP'] and row['MATCHUP'].split(' vs. ')[0] == row['TEAM_ABBREVIATION'] else None, axis=1)
    df['VISITOR_TEAM_NAME'] = df.apply(lambda row: row['TEAM_NAME'] if ' vs. ' in row['MATCHUP'] and row['MATCHUP'].split(' vs. ')[1] == row['TEAM_ABBREVIATION'] else row['TEAM_NAME'] if '@' in row['MATCHUP'] and row['MATCHUP'].split(' @ ')[0] == row['TEAM_ABBREVIATION'] else None, axis=1)

    # Group by GAME_ID and get the first occurrence of each required field
    final_df = df.groupby('GAME_ID').agg({
        'GAME_DATE': 'first',
        'SEASON_ID': 'first',
        'HOME_TEAM_ID': 'first',
        'VISITOR_TEAM_ID': 'first',
        'HOME_TEAM_NAME': 'first',
        'VISITOR_TEAM_NAME': 'first'
    }).reset_index()

    final_df = final_df[final_df["VISITOR_TEAM_ID"].isin([1610612748, 1610612747, 1610612738, 1610612743 ,1610612746 ,1610612745 ,1610612761 ,1610612749, 1610612760, 1610612762 ,1610612742 ,1610612753, 1610612757, 1610612754, 1610612755, 1610612751 ,1610612763, 1610612756 ,1610612740, 1610612764, 1610612758, 1610612759 ,1610612765 ,1610612752 ,1610612737 ,1610612766 ,1610612741 ,1610612750 ,1610612739, 1610612744])]
    final_df["VISITOR_TEAM_ID"] = final_df["VISITOR_TEAM_ID"].astype("int64")
    final_df = final_df[final_df["HOME_TEAM_ID"].isin([1610612748, 1610612747, 1610612738, 1610612743 ,1610612746 ,1610612745 ,1610612761 ,1610612749, 1610612760, 1610612762 ,1610612742 ,1610612753, 1610612757, 1610612754, 1610612755, 1610612751 ,1610612763, 1610612756 ,1610612740, 1610612764, 1610612758, 1610612759 ,1610612765 ,1610612752 ,1610612737 ,1610612766 ,1610612741 ,1610612750 ,1610612739, 1610612744])]
    rows_to_drop = final_df[final_df['HOME_TEAM_NAME'] == final_df['VISITOR_TEAM_NAME']].index
    final_df = final_df.drop(rows_to_drop)
    return final_df

def get_gamelogs(home_team_id,away_team_id, season):
    gamelog_home = teamgamelog.TeamGameLog(team_id=home_team_id, season=season)
    log_home = gamelog_home.get_data_frames()[0]
    warnings.filterwarnings("ignore", message="Could not infer format, so each element will be parsed individually")
    log_home['GAME_DATE'] = pd.to_datetime(log_home['GAME_DATE'])
    
    gamelog_away = teamgamelog.TeamGameLog(team_id=away_team_id, season=season)
    log_away = gamelog_away.get_data_frames()[0]
    warnings.filterwarnings("ignore", message="Could not infer format, so each element will be parsed individually")
    log_away['GAME_DATE'] = pd.to_datetime(log_away['GAME_DATE'])
    
    merged_df = pd.merge(log_home, log_away, on='Game_ID', suffixes=('_team1', '_team2'))

    # Reorder the columns to have all columns for team1 followed by team2
    columns = ['Game_ID'] + [col for col in merged_df.columns if col.endswith('_team1')] + [col for col in merged_df.columns if col.endswith('_team2')]
    merged_df = merged_df[columns]

    
    return merged_df

def get_logs(teamid, season):
    gamelog_home = teamgamelog.TeamGameLog(team_id=teamid, season=season)
    log_home = gamelog_home.get_data_frames()[0]
    warnings.filterwarnings("ignore", message="Could not infer format, so each element will be parsed individually")
    log_home['GAME_DATE'] = pd.to_datetime(log_home['GAME_DATE'])
    return log_home


def get_team_stats(df, end_date, games=10):
    
    filtered_games = []
    # Filter games that occurred before the end date and limit to the most recent 'games' games
    filtered_games = df[df['GAME_DATE'] < end_date]
    
    if filtered_games.shape[0] > 10:
        filtered_games = filtered_games.head(10)
        
    
    # If no games are found, return None
    if filtered_games.empty:
        print("empty")
        return None
    
    print(filtered_games.PTS)
    # Initialize an empty dictionary to hold the statistics
    stats = []
    
    #stats["PTS"] = filtered_games["PTS"]
    stats['win%'] = filtered_games['WL'].value_counts(normalize=True).get('W', 0)
    stats['efg%'] = (filtered_games['FGM'].sum() + 0.5 * filtered_games['FG3M'].sum()) / filtered_games['FGA'].sum()
    stats['fta_rate'] = filtered_games['FTA'].sum() / filtered_games['FGA'].sum()
    stats['tov%'] = filtered_games['TOV'].sum() / (filtered_games['FGA'].sum() + 0.44 * filtered_games['FTA'].sum() + filtered_games['TOV'].sum())
    stats['oreb%'] = filtered_games['OREB'].sum() / (filtered_games['DREB'].sum())
    stats['offrtg'] = filtered_games['PTS'].sum()  / 0.96*((filtered_games['FGA'].sum() )+(filtered_games['TOV'].sum()  )+0.44*(filtered_games['FTA'].sum() )-(filtered_games['OREB'].sum() ))
    stats['pace'] = 0.96*((filtered_games['FGA'].sum() )+(filtered_games['TOV'].sum()  )+0.44*(filtered_games['FTA'].sum() )-(filtered_games['OREB'].sum() )) / (48 * (filtered_games['MIN'].sum()  / 240))
    stats["ppg"] = filtered_games["PTS"].mean()
    
    return stats


In [635]:
columns = ['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
LogsSeason = pd.DataFrame(columns=columns)

for id in id_list:
    df = get_logs(id, '2023-24')
    LogsSeason = pd.concat([LogsSeason, df], ignore_index=True)

/var/folders/1h/w3mjcszn60577hhjww35tglh0000gn/T/ipykernel_48953/3625051118.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  LogsSeason = pd.concat([LogsSeason, df], ignore_index=True)


In [637]:
LogsSeason
    

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612737,0022301188,2024-04-14,ATL @ IND,L,36,46,0.439,240,39,...,0.926,9,23,32,25,6,5,15,12,115
1,1610612737,0022301178,2024-04-12,ATL @ MIN,L,36,45,0.444,240,40,...,0.680,9,31,40,23,4,1,14,25,106
2,1610612737,0022301159,2024-04-10,ATL vs. CHA,L,36,44,0.450,240,43,...,0.625,7,31,38,35,7,2,16,20,114
3,1610612737,0022301147,2024-04-09,ATL vs. MIA,L,36,43,0.456,290,45,...,0.786,17,42,59,28,13,2,15,23,111
4,1610612737,0022301130,2024-04-06,ATL @ DEN,L,36,42,0.462,240,37,...,0.828,11,27,38,30,8,2,16,18,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,1610612766,0022300133,2023-11-04,CHA @ IND,W,2,3,0.400,240,47,...,1.000,12,29,41,23,5,10,13,20,125
2456,1610612766,0022300122,2023-11-01,CHA @ HOU,L,1,3,0.250,240,45,...,0.720,15,29,44,21,9,8,16,19,119
2457,1610612766,0022300101,2023-10-30,CHA vs. BKN,L,1,2,0.333,240,47,...,0.950,8,31,39,33,8,5,9,23,121
2458,1610612766,0022300077,2023-10-27,CHA vs. DET,L,1,1,0.500,240,33,...,0.765,11,30,41,20,13,7,18,23,99


In [648]:
joined_df = LogsSeason.merge(LogsSeason, on='Game_ID',suffixes=('_1', '_2'))
joined_df = joined_df.drop_duplicates(subset='Game_ID', keep='first')
joined_df  = joined_df.drop(["GAME_DATE_2"], axis=1)


In [649]:
joined_df.info

,Team_ID_1,Game_ID,GAME_DATE_1,MATCHUP_1,WL_1,W_1,L_1,W_PCT_1,MIN_1,FGM_1,...,FT_PCT_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,PTS_2
0,1610612737,0022301188,2024-04-14,ATL @ IND,L,36,46,0.439,240,39,...,0.926,9,23,32,25,6,5,15,12,115
2,1610612737,0022301178,2024-04-12,ATL @ MIN,L,36,45,0.444,240,40,...,0.680,9,31,40,23,4,1,14,25,106
4,1610612737,0022301159,2024-04-10,ATL vs. CHA,L,36,44,0.450,240,43,...,0.625,7,31,38,35,7,2,16,20,114
6,1610612737,0022301147,2024-04-09,ATL vs. MIA,L,36,43,0.456,290,45,...,0.786,17,42,59,28,13,2,15,23,111
8,1610612737,0022301130,2024-04-06,ATL @ DEN,L,36,42,0.462,240,37,...,0.828,11,27,38,30,8,2,16,18,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576,1610612764,0022300009,2023-11-10,WAS vs. CHA,L,2,6,0.250,240,43,...,0.850,7,29,36,28,4,6,15,23,117
4578,1610612764,0022300157,2023-11-08,WAS @ CHA,W,2,5,0.286,240,47,...,0.800,13,27,40,26,10,6,10,26,132
4628,1610612765,0022300933,2024-03-11,DET vs. CHA,W,11,53,0.172,240,41,...,0.917,9,42,51,26,8,5,12,19,114
4668,1610612765,0022300619,2024-01-24,DET vs. CHA,W,5,39,0.114,240,43,...,0.600,5,39,44,26,4,0,11,17,113


In [583]:
teams_list = teams.get_teams()
seasons = [ '2019-20','2020-21', '2021-22', '2022-23', '2023-24']
all_games = []
game_data = []

for season in seasons:
    games = get_games(season)
    games = reformat_dataframe(games)
    for index, game in games.iterrows():
        logs = get_gamelogs(game['HOME_TEAM_ID'], season)
        home_stats = get_team_stats(logs,  game['GAME_DATE']) 
        logs = get_gamelogs(game['VISITOR_TEAM_ID'], season)
        away_stats = get_team_stats( logs, game['GAME_DATE'])
        if home_stats and away_stats:
            for stat in home_stats:
                game_data[f'home_{stat}'] = home_stats[stat]
                game_data[f'away_{stat}'] = away_stats[stat]
                
            all_games.append(game_data)

df = pd.DataFrame(all_games)
df.to_csv('nba_games_stats.csv', index=False)
print("Data saved to nba_games_stats.csv")

8     131
9     103
10    120
11    109
12    136
13    132
14    102
15    124
16    103
17    133
Name: PTS, dtype: int64


TypeError: list indices must be integers or slices, not str